In [ ]:

def carregar_sequencia(filename):
    from Bio import SeqIO

    try:    
        # Lemos o ficheiro .fasta
        seq_proteina = SeqIO.read(open(filename),format='fasta')

        sp = seq_proteina.seq # Isolamos a sequência
        return sp
    
    except FileNotFoundError:
        print('Ficheiro FASTA não existe. Apresentar um ficheiro FASTA válido.')




In [ ]:
filename = 'blast_results.xml'

def blast_parse(filename, lim = None, ethreshold = 0.05):
    
    from Bio.Blast import NCBIXML

    handle = open(filename)
    blast_res = NCBIXML.parse(handle)
    blast_records = list(blast_res)

    num = 1
    for record in blast_records:

        if lim == None: lim = len(record.alignments)    

        for aligment in record.alignments[0:lim]:
            #print(aligment)
            
            for hsp in aligment.hsps:
                #print(hsp)
                
                if hsp.expect < ethreshold:
                    print('****Alinhamento {} ****'.format(num))
                    print('Sequência:',aligment.title)
                    print('Tamanho da Sequência:',aligment.length)
                    print('e-value:',hsp.expect)
                    print(hsp.query[0:40]+'...')
                    print(hsp.match[0:40]+'...')
                    print(hsp.sbjct[0:40]+'...')
                    print()
                    num += 1    


In [ ]:
def analise_blast(email, fastafile, xmlfile, lim = None, ethreshold = 0.05):
    '''
    Função que carrega uma sequência de aminoácidos a partir de ficheiro fasta e realiza
    uma análise blast caso esta ainda não esteja feita ou então faz parse de blast records
    até ao limite definido.

    email : str
        parâmetro de email para utilização do serviço NCBIWWW

    fastafile : str
        caminho para ficheiro fasta com a sequência da proteína

    xmlfile : str
        caminho para o ficheiro XML com blastrecords caso já esteja previamente realizada

    lim : int
        limite de resultados que queremos visualizar dos resultados do blast
        por defeito não está definido

    e-threshold : float
        valor-limite de e-value a que queremos delimitar a nossa análise blast
        por defeito está definido 0.05
    
    '''
    import os
    from Bio.Blast import NCBIWWW
    
    NCBIWWW.email = email

    if not fastafile.endswith('.fasta'): raise ValueError('Ficheiro .fasta inválido')  

    try:
        sequencia_proteina = carregar_sequencia(fastafile)
    except FileNotFoundError: 
        print('Ficheiro FASTA não existe. Apresentar um ficheiro FASTA válido.')
        print('Sugestão: Realizar um pesquisa_ncbi() com o ID da proteína na base de dados protein')
    
    # Verifica se já existe um ficheiro XML para os resultados do BLAST. Ignora a pesquisa caso já exista.
    # Para poupar tempo.
    # Se o ficheiro já existir executa o parsing do ficheiro.

    if not xmlfile.endswith('.xml'):
        xmlfile = xmlfile + '.xml'

    # Verifica a existência do ficheiro

    if os.path.isfile(xmlfile): 
        blast_parse(xmlfile,lim, ethreshold)
        
    
    else: 
    # Executa o BLAST da Proteína
        handle = NCBIWWW.qblast('blastp','swissprot',sequencia_proteina)
        with open(xmlfile, 'w') as _:
            _.write(handle.read())

        blast_parse(xmlfile,lim,ethreshold)

        # Retorna ao working directory anterior
        

        

        print(f'Ficheiro {xmlfile} gravado.')
        
        handle.close()